<a href="https://colab.research.google.com/github/LamuGit/FinText-Data/blob/main/Copy_of_Random_Forest_Meta_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install xlsxwriter
!pip install pip install PyPortfolioOpt
import pandas as pd
from xlsxwriter import Workbook
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import PyPortfolioOpt
import pypfopt
from pypfopt import black_litterman, risk_models
from pypfopt import BlackLittermanModel, plotting
from pypfopt import EfficientFrontier, objective_functions




# Load the files
df1 = pd.read_excel('/Extracted_data_sentiment_Lamu_FinBert_ProsusAI.xlsx')
df2 = pd.read_excel('/Extracted_data_sentiment_Lamu_Finbert_yiyanghkust.xlsx')
df3 = pd.read_excel('/Extracted_data_sentiment_Lamu_distilroberta_mrm8488.xlsx')

# Define a function to calculate the sentiment score for each sentence
def calculate_sentiment_score(row):
    sentiment_score = row['Positive Probability'] - row['Negative Probability'] + (row['Neutral Probability'] * 0.5)
    return sentiment_score

# Apply the function to each row of the dataframes
df1['Sentiment Score'] = df1.apply(calculate_sentiment_score, axis=1)
df2['Sentiment Score'] = df2.apply(calculate_sentiment_score, axis=1)
df3['Sentiment Score'] = df3.apply(calculate_sentiment_score, axis=1)

# Rename the column 'Sentiment Score' in df3 to 'Sentiment Score_llm3'
df3 = df3.rename(columns={'Sentiment Score': 'Sentiment Score_llm3'})

# Rename the column 'Sentiment Score' in df1 and df2
df1 = df1.rename(columns={'Sentiment Score': 'Sentiment Score_llm1'})
df2 = df2.rename(columns={'Sentiment Score': 'Sentiment Score_llm2'})

# Merge the dataframes on the Sentence column
merged_df = pd.merge(df1, df2, on='Sentence', how='inner')
merged_df = pd.merge(merged_df, df3, on='Sentence', how='inner')

# Define the feature columns (sentiment scores from each model)
X = merged_df[['Sentiment Score_llm1', 'Sentiment Score_llm2', 'Sentiment Score_llm3']]

# Define the target column (average sentiment score)
y = merged_df[['Sentiment Score_llm1', 'Sentiment Score_llm2', 'Sentiment Score_llm3']].mean(axis=1)

# Scale the data using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Define hyperparameter grid for Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
 'max_depth': [None, 5, 10],
 'min_samples_split': [2, 5, 10]
}

# Perform grid search to find optimal hyperparameters
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Train a Random Forest model on the training data using optimal hyperparameters
rf = RandomForestRegressor(**grid_search.best_params_)
rf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf.predict(X_test)

# Evaluate the model using multiple metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}')

# Save the results to a new spreadsheet
results_df = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})
results_df.to_excel('results.xlsx', index=False, engine='xlsxwriter')

# Save the model performance metrics to a new spreadsheet
metrics_df = pd.DataFrame({
    'Metric': ['MSE', 'MAE', 'R2'],
    'Value': [mse, mae, r2]
})
metrics_df.to_excel('metrics.xlsx', index=False, engine='xlsxwriter')

# Use the trained model to make predictions on the entire dataset
merged_df['Meta Model Score'] = rf.predict(X_scaled)

# Save the resulting dataframe to a new spreadsheet
merged_df.to_excel('meta_model_scores_update.xlsx', index=False, engine='xlsxwriter')


###################################################
#get stock prices
##################################################
# Define the stock symbols

# Load the meta model scores
meta_model_scores = pd.read_excel('meta_model_scores_update.xlsx')

# Get the stock codes
stock_codes = meta_model_scores['Stock'].unique()

# Get the top 10 stock codes
top_10_stock_codes = meta_model_scores['Stock'].value_counts().head(10).index

# Create an empty list to store the stock price data
stock_price_data_list = []

# Loop through each stock code and get the stock price data for 2016
for stock_code in top_10_stock_codes:
    ticker = yf.Ticker(stock_code)
    hist = ticker.history(start='2016-01-01', end='2024-07-31')
    hist['Stock'] = stock_code
    hist.reset_index(inplace=True)
    hist.rename(columns={'index': 'Date'}, inplace=True)
    stock_price_data_list.append(hist)

# Concatenate the stock price data into a single DataFrame
stock_price_data = pd.concat(stock_price_data_list)

# Convert 'Date' column to datetime objects with utc=True to handle timezone-aware datetimes
stock_price_data['Date'] = pd.to_datetime(stock_price_data['Date'], utc=True)

# Now convert to timezone-naive if necessary
stock_price_data['Date'] = stock_price_data['Date'].dt.tz_localize(None)

# Save the stock price data to a new Excel file
stock_price_data.to_excel('stock_price_data_2016to2024.xlsx', index=False, engine='xlsxwriter')

###############################################################
#Black-Litterman v1
###############################################################
tickers = top_10_stock_codes.tolist()
#tickers = ["MSFT", "AAPL", "ORCL", "BAC", "JPM", "BHP", "TSLA", "AMZN", "CSCO", "PEP"]
ohlc = yf.download(tickers, period="max")
prices = ohlc["Adj Close"]
prices.tail()
#market prices
market_prices = yf.download("SPY", period="max")["Adj Close"]
market_prices.head()
#market cap
mcaps = {}
for t in tickers:
    stock = yf.Ticker(t)
    mcaps[t] = stock.info["marketCap"]
mcaps

#Black litterman prior
S = risk_models.CovarianceShrinkage(prices).ledoit_wolf()
delta = black_litterman.market_implied_risk_aversion(market_prices)
delta
plotting.plot_covariance(S, plot_correlation=True);
market_prior = black_litterman.market_implied_prior_returns(mcaps, delta, S)
market_prior
market_prior.plot.barh(figsize=(10,5));

#Views
# Map meta model scores to expected returns using a linear transformation for each stock
absolute_return_views = {}
for stock_code in top_10_stock_codes:
    stock_data = stock_price_data[stock_price_data['Stock'] == stock_code]
    stock_returns = stock_data['Close'].pct_change().mean()
    stock_meta_model_scores = meta_model_scores[meta_model_scores['Stock'] == stock_code]['Meta Model Score']
    min_meta_model_score = stock_meta_model_scores.min()
    max_meta_model_score = stock_meta_model_scores.max()
    min_stock_return = stock_returns.min()
    max_stock_return = stock_returns.max()

    # Apply linear transformation
    expected_return = (stock_meta_model_scores.mean() - min_meta_model_score) / (max_meta_model_score - min_meta_model_score) * (max_stock_return - min_stock_return) + min_stock_return
    absolute_return_views[stock_code] = expected_return*100

# Convert absolute return views to a pandas Series
absolute_return_views = pd.Series(absolute_return_views)



#posterior returns
bl = BlackLittermanModel(S, pi=market_prior, absolute_views=absolute_return_views)
ret_bl = bl.bl_returns()
ret_bl

rets_df = pd.DataFrame([market_prior, ret_bl, pd.Series(absolute_return_views)],
             index=["Prior", "Posterior", "Views"]).T
rets_df
rets_df.plot.bar(figsize=(12,8));

#posterior covariance
S_bl = bl.bl_cov()
S_bl

ef = EfficientFrontier(ret_bl, S_bl)
ef.add_objective(objective_functions.L2_reg)
ef.max_sharpe()
weights = ef.clean_weights()
weights


###############################################################
#Black-Litterman v2
###############################################################

# Load the meta model scores
meta_model_scores = pd.read_excel('meta_model_scores_update.xlsx')

# Concatenate the stock price data into a single DataFrame
stock_price_data = pd.concat(stock_price_data_list)

# Calculate the mean and covariance of the stock prices
mean_returns = stock_price_data.groupby('Stock')['Close'].mean()
cov_matrix = stock_price_data.pivot_table(index='Date', columns='Stock', values='Close').cov()

# Define the prior distribution
prior_mean = mean_returns
prior_cov = cov_matrix

# Define the view distribution
view_mean = meta_model_scores[meta_model_scores['Stock'].isin(top_10_stock_codes)].groupby('Stock')['Meta Model Score'].mean()
view_cov = np.diag(meta_model_scores[meta_model_scores['Stock'].isin(top_10_stock_codes)].groupby('Stock')['Meta Model Score'].std())


# Define the Black-Litterman model parameters
tau = 0.05  # confidence in the views..0.25 or 1 other vales
omega = np.diag(np.ones(len(top_10_stock_codes)))  # view covariance matrix

# Define the Black-Litterman model function
def black_litterman(prior_mean, prior_cov, view_mean, omega, tau):
    posterior_mean = (np.linalg.inv(prior_cov) + tau * np.linalg.inv(omega)) @ (prior_mean + tau * view_mean)
    posterior_cov = np.linalg.inv(np.linalg.inv(prior_cov) + tau * np.linalg.inv(omega))
    return posterior_mean, posterior_cov

# Run the Black-Litterman model
posterior_mean, posterior_cov = black_litterman(prior_mean, prior_cov, view_mean, omega, tau)

# Print the results
print("Prior Distribution:")
print("Mean:", prior_mean)
print("Covariance:", prior_cov)

print("\nView Distribution:")
print("Mean:", view_mean)
print("Covariance:", view_cov)

print("\nPosterior Distribution:")
print("Mean:", posterior_mean)
print("Covariance:", posterior_cov)

# Convert covariance matrices to lists of lists for DataFrame compatibility
prior_cov_list = prior_cov.values.tolist()
view_cov_list = view_cov.tolist()  # Assuming view_cov is a numpy array
posterior_cov_list = posterior_cov.tolist()

# Save the results to a new Excel file
results_df = pd.DataFrame({
    'Prior Mean': prior_mean,
    'Prior Covariance': prior_cov_list,  # Use lists of lists
    'View Mean': view_mean,
    'View Covariance': view_cov_list,
    'Posterior Mean': posterior_mean,
    'Posterior Covariance': posterior_cov_list
})
results_df.to_excel('black_litterman_results.xlsx', index=False, engine='xlsxwriter')

ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install


ModuleNotFoundError: No module named 'PyPortfolioOpt'

In [61]:
# Define the quarterly rebalancing function
def quarterly_rebalancing(prices, meta_model_scores, market_prior, S, mcaps, absolute_return_views):
    # Initialize the portfolio weights
    weights = [1/len(tickers) for _ in tickers]

    # Initialize the portfolio performance
    portfolio_performance = []

    # Initialize the equally weighted portfolio performance
    equally_weighted_performance = []

    # Initialize the cap-weighted portfolio performance
    cap_weighted_performance = []

    # Iterate over the quarters
    for quarter in range(0, len(prices), 63):
        # Get the prices for the current quarter
        quarter_prices = prices.iloc[quarter:quarter+63]

        # Get the meta model scores for the current quarter
        quarter_meta_model_scores = meta_model_scores.iloc[quarter:quarter+63]

        # Map meta model scores to expected returns using a linear transformation for each stock
        absolute_return_views = {}
        for stock_code in top_10_stock_codes:
            stock_data = stock_price_data[stock_price_data['Stock'] == stock_code]
            stock_returns = stock_data['Close'].pct_change().mean()
            stock_meta_model_scores = quarter_meta_model_scores[quarter_meta_model_scores['Stock'] == stock_code]['Meta Model Score']
            min_meta_model_score = stock_meta_model_scores.min()
            max_meta_model_score = stock_meta_model_scores.max()
            min_stock_return = stock_returns.min()
            max_stock_return = stock_returns.max()

            # Apply linear transformation
            expected_return = (stock_meta_model_scores.mean() - min_meta_model_score) / (max_meta_model_score - min_meta_model_score) * (max_stock_return - min_stock_return) + min_stock_return
            absolute_return_views[stock_code] = expected_return*100

        # Convert absolute return views to a pandas Series
        absolute_return_views = pd.Series(absolute_return_views)

        # Posterior returns
        bl = BlackLittermanModel(S, pi=market_prior, absolute_views=absolute_return_views)
        ret_bl = bl.bl_returns()

        # Posterior covariance
        S_bl = bl.bl_cov()

        # Efficient frontier
        ef = EfficientFrontier(ret_bl, S_bl)
        ef.add_objective(objective_functions.L2_reg)
        ef.max_sharpe()
        weights = ef.clean_weights()

        # Calculate the portfolio performance for the current quarter
        portfolio_performance.append(np.sum(weights * quarter_prices.iloc[-1]))

        # Calculate the equally weighted portfolio performance for the current quarter
        equally_weighted_performance.append(np.sum([1/len(tickers) for _ in tickers] * quarter_prices.iloc[-1]))

        # Calculate the cap-weighted portfolio performance for the current quarter
        cap_weighted_performance.append(np.sum([mcaps[ticker] / sum(mcaps.values()) for ticker in tickers] * quarter_prices.iloc[-1]))

    # Plot the portfolio performance graph
    plt.plot(portfolio_performance, label='Black-Litterman Portfolio')
    plt.plot(equally_weighted_performance, label='Equally Weighted Portfolio')
    plt.plot(cap_weighted_performance, label='Cap-Weighted Portfolio')
    plt.legend()
    plt.show()

# Call the quarterly rebalancing function
quarterly_rebalancing(prices, meta_model_scores, market_prior, S, mcaps, absolute_return_views)

ERROR in LDL_factor: Error in KKT matrix LDL factorization when computing the nonzero elements. The problem seems to be non-convex
ERROR in osqp_setup: KKT matrix factorization.
The problem seems to be non-convex.


<ipython-input-61-dddf9832f30c>:35: RuntimeWarning: invalid value encountered in scalar divide
  expected_return = (stock_meta_model_scores.mean() - min_meta_model_score) / (max_meta_model_score - min_meta_model_score) * (max_stock_return - min_stock_return) + min_stock_return
/usr/local/lib/python3.10/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


SolverError: Workspace allocation error!

In [69]:
# Define the quarterly rebalancing function
def quarterly_rebalancing(prices, meta_model_scores, market_prior, S, mcaps, absolute_return_views):
    # Initialize the portfolio weights
    weights = [1/len(tickers) for _ in tickers]

    # Initialize the portfolio performance
    portfolio_performance = []

    # Initialize the equally weighted portfolio performance
    equally_weighted_performance = []

    # Initialize the cap-weighted portfolio performance
    cap_weighted_performance = []

    # Iterate over the quarters
    for quarter in range(0, len(prices), 63):
        # Get the prices for the current quarter
        quarter_prices = prices.iloc[quarter:quarter+63]

        # Get the meta model scores for the current quarter
        quarter_meta_model_scores = meta_model_scores.iloc[quarter:quarter+63]

        # Map meta model scores to expected returns using a linear transformation for each stock
        absolute_return_views = {}
        for stock_code in top_10_stock_codes:
            stock_data = stock_price_data[stock_price_data['Stock'] == stock_code]
            stock_returns = stock_data['Close'].pct_change().mean()
            stock_meta_model_scores = quarter_meta_model_scores[quarter_meta_model_scores['Stock'] == stock_code]['Meta Model Score']
            min_meta_model_score = stock_meta_model_scores.min()
            max_meta_model_score = stock_meta_model_scores.max()
            min_stock_return = stock_returns.min()
            max_stock_return = stock_returns.max()

            # Apply linear transformation
            expected_return = (stock_meta_model_scores.mean() - min_meta_model_score) / (max_meta_model_score - min_meta_model_score) * (max_stock_return - min_stock_return) + min_stock_return
            absolute_return_views[stock_code] = expected_return*100

        # Convert absolute return views to a pandas Series
        absolute_return_views = pd.Series(absolute_return_views)

        # Posterior returns
        bl = BlackLittermanModel(S, pi=market_prior, absolute_views=absolute_return_views)
        ret_bl = bl.bl_returns()

        # Posterior covariance
        S_bl = bl.bl_cov()

        # Efficient frontier
        ef = EfficientFrontier(ret_bl, S_bl)
        ef.add_objective(objective_functions.L2_reg)
        # Increase the memory available to the solver
        ef._solver_options = {"max_iter": 1000000}  # Adjust the value as needed
        ef.max_sharpe()
        weights = ef.clean_weights()

        # Calculate the portfolio performance for the current quarter
        portfolio_performance.append(np.sum(weights * quarter_prices.iloc[-1]))

        # Calculate the equally weighted portfolio performance for the current quarter
        equally_weighted_performance.append(np.sum([1/len(tickers) for _ in tickers] * quarter_prices.iloc[-1]))

        # Calculate the cap-weighted portfolio performance for the current quarter
        cap_weighted_performance.append(np.sum([mcaps[ticker] / sum(mcaps.values()) for ticker in tickers] * quarter_prices.iloc[-1]))

# Plot the portfolio performance graph
    plt.plot(portfolio_performance, label='Black-Litterman Portfolio')
    plt.plot(equally_weighted_performance, label='Equally Weighted Portfolio')